In [16]:
import json
import torch
import numpy as np
import matplotlib.pyplot as plt

from sentence_transformers import SentenceTransformer, util, CrossEncoder
from tqdm import tqdm

In [17]:
def contains_both_words(sentence):
    lower_sentence = sentence.lower()  # Convert the entire sentence to lowercase
#     return ('there' in lower_sentence and 'no' in lower_sentence) or ('no' in lower_sentence and 'available' in lower_sentence) or ('none' in lower_sentence)
    return ('there' in lower_sentence and 'no' in lower_sentence)

In [34]:
model = CrossEncoder('cross-encoder/ms-marco-MiniLM-L-12-v2', max_length=512)

total_accs = []
for chunk_idx in range(19) :
    print('###', chunk_idx)
    chunk_accs = []
    
    for i in range(10) :

        data_list = []
        for j in range(i+1) :
            with open(f"/scratch/project/retrieval-augmented-generation-explanation-calibration"\
            "/Ours-framework/results/results-0923-ours-rerank-newreader-ltm-bm25-t20-one_by_one-gpt-3.5-turbo"\
            f"/chunk-{chunk_idx}/idx-{j}/open_wikitq_validation.json") as file:    
                data = json.load(file)
                data_list.append(data)

        preds = []
        for sample_k in data_list[0].keys() :
            if sample_k == 'EM_Acc' :
                continue

            scores, sql_list = [], []
            question = data_list[0][sample_k]['question']

            for data in data_list :

                if len(data[sample_k]['pred_result']) != 0 and \
                data[sample_k]['subtable'] is not None and \
                not contains_both_words(data[sample_k]['pred_result'][0]):
                    score = data[sample_k]['pred_score']
                    scores.append(score)
                    sql_list.append(data[sample_k]['sql_prog_used'])

            if len(scores) == 0 :
                preds.append(False)
            else :
                sim_score = model.predict([(question, sql_used) for sql_used in sql_list])
                preds.append(scores[np.argmax(sim_score)])

        acc = (torch.tensor(preds).sum()/len(preds)).item()
        print(i, acc)
        chunk_accs.append(acc)
        
    total_accs.append(chunk_accs)
        
print('### final result')
total_accs = torch.tensor(total_accs)
total_accs.mean(dim=0)

### 0
0 0.46875
1 0.484375
2 0.5
3 0.546875
4 0.5546875
5 0.5546875
6 0.5390625
7 0.5390625
8 0.546875
9 0.5390625
### 1
0 0.5078125
1 0.5546875
2 0.5625
3 0.5703125
4 0.5546875
5 0.5625
6 0.5625
7 0.5390625
8 0.546875
9 0.546875
### 2
0 0.4375
1 0.4765625
2 0.484375
3 0.484375
4 0.484375
5 0.46875
6 0.453125
7 0.46875
8 0.46875
9 0.46875
### 3
0 0.4296875
1 0.4375
2 0.46875
3 0.4921875
4 0.5
5 0.5078125
6 0.515625
7 0.5234375
8 0.5390625
9 0.5390625
### 4
0 0.53125
1 0.5625
2 0.5703125
3 0.59375
4 0.5703125
5 0.5703125
6 0.578125
7 0.578125
8 0.578125
9 0.5859375
### 5
0 0.53125
1 0.546875
2 0.5625
3 0.5625
4 0.5703125
5 0.5703125
6 0.59375
7 0.6015625
8 0.6015625
9 0.6015625
### 6
0 0.453125
1 0.53125
2 0.5546875
3 0.5546875
4 0.5703125
5 0.578125
6 0.5703125
7 0.5703125
8 0.5703125
9 0.5859375
### 7
0 0.4765625
1 0.5078125
2 0.546875
3 0.546875
4 0.5390625
5 0.5234375
6 0.53125
7 0.53125
8 0.53125
9 0.5234375
### 8
0 0.5234375
1 0.5625
2 0.59375
3 0.6015625
4 0.609375
5 0.6015625
6 

tensor([0.4910, 0.5238, 0.5407, 0.5535, 0.5563, 0.5576, 0.5609, 0.5584, 0.5621,
        0.5654])